In [1]:
#Created by Kenneth Lulie
#This code is to primarily pull Form 8-Ks from the SEC Website to be used with BERT
#Some parts were pulled from  https://github.com/markbabbe/BERT-Stock-Prediction-Using-NLP

# This was added to deal with API timeouts

import signal
class Timeout():
    class TimeoutException(Exception):
        pass

    def _timeout(signum, frame):
        raise Timeout.TimeoutException()

    def __init__(self, timeout=10):
        self.timeout = timeout
        signal.signal(signal.SIGALRM, Timeout._timeout)

    def __enter__(self):
        signal.alarm(self.timeout)

    def __exit__(self, exc_type, exc_value, traceback):
        signal.alarm(0)
        return exc_type is Timeout.TimeoutException

In [2]:
#Load Libraries that will be needed
import re
import requests
from bs4 import BeautifulSoup
import datetime
import unicodedata
import requests
import pandas as pd
import numpy as np
from time import sleep
import math
from config import Config
import dateutil.relativedelta
#import pandas_market_calendars as mcal
import os
import io
import re
from tqdm import tqdm
import gc
import ast


#Makes a class to hold the return of these functions
class SEC_Extractor:
    #feed in a cik and a ticker
    def get_doc_links(cik,ticker):
        
        #This is a function to get the list of 8-ks we will web-scrape later
        try:
            base_url = "https://www.sec.gov/cgi-bin/browse-edgar"
            inputted_cik = cik
            #Payload is for interacting with the API
            payload = {
                "action" : "getcompany",
                "CIK" : inputted_cik,
                "type" : "8-K",
                "output":"xml",
                #Dateb stand for 'datebefore'
                #Changing from 20180401 to 2020-02-20
                "dateb" : "20200220",
            }
            #sec_response will hold the response from the SEC, which will be an 8-k
            sec_response = requests.get(url=base_url,params=payload)
            # soup is the parsed text of the response, using the lxml parser
            soup = BeautifulSoup(sec_response.text,'lxml')
            #Changed from url_link to url_link to be more descriptive
            #find the link by searching for filinghref
            url_link = soup.findAll('filinghref')
            html_list = []
            #Run a for loop that fixes a formatting error, and appends the link to the html_list
            for link in url_link:
                link = link.string
                if link.split(".")[len(link.split("."))-1] == 'htm':
                    txtlink = link + "l"
                    #then append it to text link
                    html_list.append(txtlink)
                    
                    
            doc_list = []
            doc_name_list = []
            #run a for loop that for each link in the html list, replaces the -index.html with .txt
            #this converts the link to a link to the text document which is what we want
            for k in range(len(html_list)):
                txt_doc = html_list[k].replace("-index.html",".txt")
                #This splits the link at the /, and only keeps the value at the end
                #Create a variable to call the doc name
                doc_name = txt_doc.split("/")[-1]
                #append the varible txt doc, to the doc list
                doc_list.append(txt_doc)
                #append the name of the document, to the list document name link.
                doc_name_list.append(doc_name)
                
                
                # Create dataframe of CIK, doc name, and txt link
                #Creates a dataframe, the cik is the cik from the html list
                #the ticket is the ticket from the html_list
                #THe txt link is from the 'doc list' this is where to pull the text file from
                #The doc name is the name of the text file we are pulling
            df = pd.DataFrame(
                {
                "cik" : [cik]*len(html_list),
                "ticker" : [ticker]*len(html_list),
                "txt_link" : doc_list,
                "doc_name": doc_name_list
                }
            )
        except requests.exceptions.ConnectionError:
                sleep(.1)
        return df

    # Extracts text and submission datetime from document link
    def extract_text(link):
        
      check = 0
      with Timeout(20):
        try:
            #Feed in the text link from the dataframe we created earlier
            #r is the payload received
                r = requests.get(link)
                print("Done Link")
                
                #Changed parser from html5lib to lxml to run faster
                filing = BeautifulSoup(r.content,"lxml",from_encoding="ascii")
                
                #find the field 'acceptance-datetime", and then pull the next 14 characters'
                try:
                    submission_dt = filing.find("acceptance-datetime").string[:14]
                except AttributeError:
                    #IF submission date is 2018, throw an error
                    
                    #Flag the submission date as 2050 if there is any issue finding the submission date
                    submission_dt = "2050501100000"
                print("Done Date")

                submission_dt = datetime.datetime.strptime(submission_dt,"%Y%m%d%H%M%S")
 
                #Do text parsing
                #convert tables to text
                #Convert sections to unicode
                for section in filing.findAll("html"):
                    try:
                        for table in section("table"):
                            table.decompose()
                        section = unicodedata.normalize("NFKD",section.text)
                        #Remove special characters just as new lines
                        section = section.replace("\t"," ").replace("\n"," ").replace("/s"," ").replace("\'","'")            
                    except AttributeError:
                        section = str(section.encode('utf-8'))
                filing = "".join((section))
        except requests.exceptions.ConnectionError:
                sleep(10)
        sleep(.1)
        print ("Done filing")
        return filing, submission_dt
        check = 1
      print(check)
      if check == 0:
        return ("Passed", "Passed")
      
    def extract_item_no(document):
      if document != "Passed":
        pattern = re.compile("Item+ +\d+[\:,\.]+\d+\d")
        item_list = re.findall(pattern,document)
        return item_list
      else:
        return


### 2. Get S&P 500 Data

In [17]:
# Get table of the S&P 500 tickers, CIK, and industry from Wikipedia
# This is so we know which Form 8-Ks to pull
wiki_url = "https://en.wikipedia.org/wiki/List_of_S%26P_500_companies"
cik_df = pd.read_html(wiki_url,header=0,index_col=0)[0]
cik_df['GICS Sector'] = cik_df['GICS Sector'].astype("category")
cik_df['GICS Sub Industry'] = cik_df['GICS Sector'].astype("category")
cik_df.head()

,Security,SEC filings,GICS Sector,GICS Sub Industry,Headquarters Location,Date first added,CIK,Founded
Symbol,,,,,,,,
MMM,3M Company,reports,Industrials,Industrials,"St. Paul, Minnesota",NaN,66740,1902
ABT,Abbott Laboratories,reports,Health Care,Health Care,"North Chicago, Illinois",1964-03-31,1800,1888
ABBV,AbbVie Inc.,reports,Health Care,Health Care,"North Chicago, Illinois",2012-12-31,1551152,2013 (1888)
ABMD,ABIOMED Inc,reports,Health Care,Health Care,"Danvers, Massachusetts",2018-05-31,815094,1981
ACN,Accenture plc,reports,Information Technology,Information Technology,"Dublin, Ireland",2011-07-06,1467373,1989


### 3. Get List of 8K links from SEC Edgar

In [9]:
sec_ext = SEC_Extractor

In [5]:

#pull the list of S&P500 companies from Wikipedia
#uses the ticket/cik combo to pull all 8-k reports
#note this will pull up to 40 8-Ks per company, and 500 in the S&P gives us 20,000 as our total

df_list = []
company_list = cik_df['CIK'].to_dict()
#tqdm is a progress bar
for (ticker,cik) in tqdm(company_list.items()):
    df_list.append(sec_ext.get_doc_links(cik,ticker))
#this is where we pull from the doc links, to make a dataframe of what we want to pull

#axis{0/’index’, 1/’columns’}, default 0
# concaatante the list using the index
doc_links_df = pd.concat(df_list,axis=0)

#
doc_links_df = doc_links_df.set_index("ticker").join(cik_df['GICS Sector']).join(cik_df['GICS Sub Industry']).reset_index().rename(columns={"index":"ticker"})
doc_links_df.head()


100%|██████████| 505/505 [01:36<00:00,  5.25it/s]


,ticker,cik,txt_link,doc_name,GICS Sector,GICS Sub Industry
0,A,1090872.0,https://www.sec.gov/Archives/edgar/data/109087...,0001090872-20-000002.txt,Health Care,Health Care
1,A,1090872.0,https://www.sec.gov/Archives/edgar/data/109087...,0001564590-20-001714.txt,Health Care,Health Care
2,A,1090872.0,https://www.sec.gov/Archives/edgar/data/109087...,0001090872-19-000020.txt,Health Care,Health Care
3,A,1090872.0,https://www.sec.gov/Archives/edgar/data/109087...,0001564590-19-037386.txt,Health Care,Health Care
4,A,1090872.0,https://www.sec.gov/Archives/edgar/data/109087...,0001564590-19-034970.txt,Health Care,Health Care


In [12]:
#save the doc links as a csv
#save it as a pickle file

doc_links_df.to_csv("/home/kl/doc_links_df.csv")
doc_links_df.to_pickle("/home/kl/Pickles/doc_links_df.pkl")

### 4. Download 8Ks & Stock Movements

In [5]:
crawled_df = pd.read_pickle("/home/kl/Pickles/doc_links_df.pkl")

In [6]:
crawled_df.head()

,ticker,cik,txt_link,doc_name,GICS Sector,GICS Sub Industry
0,A,1090872.0,https://www.sec.gov/Archives/edgar/data/109087...,0001090872-20-000002.txt,Health Care,Health Care
1,A,1090872.0,https://www.sec.gov/Archives/edgar/data/109087...,0001564590-20-001714.txt,Health Care,Health Care
2,A,1090872.0,https://www.sec.gov/Archives/edgar/data/109087...,0001090872-19-000020.txt,Health Care,Health Care
3,A,1090872.0,https://www.sec.gov/Archives/edgar/data/109087...,0001564590-19-037386.txt,Health Care,Health Care
4,A,1090872.0,https://www.sec.gov/Archives/edgar/data/109087...,0001564590-19-034970.txt,Health Care,Health Care


In [7]:

#Due to the size of the pull, i've split it up into 10 parts
#Increment part_no up to your total number, each time it will save it to a csv
#Then you will need to combine the CSVs into one.
#Please note I had to manually drop one of the links, as the 8-k appears to have special characters that throw an error
##"https://www.sec.gov/Archives/edgar/data/885639/000089271219000344/0000892712-19-000344.txt" <- should be removed 

no_parts = 10
part_no = 1

In [10]:

crawled_df = pd.read_pickle("/home/kl/Pickles/doc_links_df.pkl")
# had to change 'chunksize' to 'chunks' to get it to run   
#Load pickle
# crawled_df = np.array_split(pd.read_pickle("Pickles/doc_links_df.pkl"),no_parts)[part_no-1]
crawled_df =  np.array_split(crawled_df, no_parts)[part_no-1]
crawled_len = len(crawled_df['txt_link'])

chunks = crawled_len
chunksize = chunks
#tqdm is for the progress bars

df_list = []
for i, df in tqdm(enumerate(np.array_split(crawled_df,chunksize))):
    print(i)
    df['text'], df['release_date'] = zip(*df['txt_link'].apply(sec_ext.extract_text))
    df['items'] = df['text'].map(sec_ext.extract_item_no)
    if not os.path.isfile("/home/kl/Data/texts_example{}.csv".format(part_no)): #If no file exists, create one with header
        df.to_csv("/home/kl/Data/texts_example{}.csv".format(part_no),chunksize=chunksize,)
    else: # else it exists so append without writing the header
        df.to_csv("/home/kl/Data/texts_example{}.csv".format(part_no),mode="a",header=False,chunksize=chunksize)       
    df_list.append(df)
    del df
df = pd.concat(df_list)
df.head()


0it [00:00, ?it/s]

0


1it [00:00,  2.43it/s]

Done Link
Done Date
Done filing
1


2it [00:00,  2.36it/s]

...

1997it [27:11,  1.75it/s]

Done Link
Done Date
Done filing
1997


1998it [27:11,  1.22it/s]

Done Link
Done Date
Done filing


,ticker,cik,txt_link,doc_name,GICS Sector,GICS Sub Industry,text,release_date,items
1998,ATO,731802.0,https://www.sec.gov/Archives/edgar/data/731802...,0000731802-17-000035.txt,Utilities,Utilities,0000731802-17-000035.txt : 20170808 0000731802...,2017-08-08 17:01:43,[]
1999,ATO,731802.0,https://www.sec.gov/Archives/edgar/data/731802...,0000731802-17-000031.txt,Utilities,Utilities,0000731802-17-000031.txt : 20170802 0000731802...,2017-08-02 17:06:34,[Item 2.02]
2000,ATO,731802.0,https://www.sec.gov/Archives/edgar/data/731802...,0001193125-17-198618.txt,Utilities,Utilities,0001193125-17-198618.txt : 20170608 0001193125...,2017-06-08 17:01:57,[]
2001,ATO,731802.0,https://www.sec.gov/Archives/edgar/data/731802...,0001193125-17-197687.txt,Utilities,Utilities,0001193125-17-197687.txt : 20170608 0001193125...,2017-06-08 08:04:05,[]
2002,ATO,731802.0,https://www.sec.gov/Archives/edgar/data/731802...,0000731802-17-000018.txt,Utilities,Utilities,0000731802-17-000018.txt : 20170503 0000731802...,2017-05-03 17:06:09,[Item 2.02]


In [ ]:
#To monitor the process for errors, i create a master file by copying the first section
master = pd.read_csv("/home/kl/Data/texts_example1.csv")
master.to_csv("/home/kl/Data/master.csv")

In [27]:
#Then I iterate through the file name to add the remaining sections
text_part = pd.read_csv("/home/kl/Data/texts_example10.csv")
text_part.to_csv("/home/kl/Data/master.csv",mode="a",header=False)

In [119]:
#Read in data
#Check for NAs
master = pd.read_csv("F:\Files from Linux\Raw Texts\master.csv")
print(master.isna().sum())

Unnamed: 0            0
Unnamed: 0.1          0
ticker                0
cik                   0
txt_link              0
doc_name              0
GICS Sector           0
GICS Sub Industry     0
text                  0
release_date          0
items                51
dtype: int64


In [120]:
print(len(master))

19973


In [121]:
master.head(100)

,Unnamed: 0,Unnamed: 0.1,ticker,cik,txt_link,doc_name,GICS Sector,GICS Sub Industry,text,release_date,items
0,0,0,A,1090872.0,https://www.sec.gov/Archives/edgar/data/109087...,0001090872-20-000002.txt,Health Care,Health Care,0001090872-20-000002.txt : 20200218 0001090872...,2020-02-18 16:09:58,"['Item 2.02', 'Item 2.02', 'Item 9.01']"
1,1,1,A,1090872.0,https://www.sec.gov/Archives/edgar/data/109087...,0001564590-20-001714.txt,Health Care,Health Care,0001564590-20-001714.txt : 20200123 0001564590...,2020-01-23 08:15:23,[]
2,2,2,A,1090872.0,https://www.sec.gov/Archives/edgar/data/109087...,0001090872-19-000020.txt,Health Care,Health Care,0001090872-19-000020.txt : 20191125 0001090872...,2019-11-25 16:07:24,"['Item 2.02', 'Item 2.02', 'Item 9.01']"
3,3,3,A,1090872.0,https://www.sec.gov/Archives/edgar/data/109087...,0001564590-19-037386.txt,Health Care,Health Care,0001564590-19-037386.txt : 20191022 0001564590...,2019-10-22 16:05:46,[]
4,4,4,A,1090872.0,https://www.sec.gov/Archives/edgar/data/109087...,0001564590-19-034970.txt,Health Care,Health Care,0001564590-19-034970.txt : 20190918 0001564590...,2019-09-18 16:30:38,[]
5,5,5,A,1090872.0,https://www.sec.gov/Archives/edgar/data/109087...,0001193125-19-245863.txt,Health Care,Health Care,0001193125-19-245863.txt : 20190916 0001193125...,2019-09-16 16:42:10,"['Item 1.01', 'Item 2.03']"
6,6,6,A,1090872.0,https://www.sec.gov/Archives/edgar/data/109087...,0001193125-19-240125.txt,Health Care,Health Care,0001193125-19-240125.txt : 20190906 0001193125...,2019-09-06 17:28:10,[]
7,7,7,A,1090872.0,https://www.sec.gov/Archives/edgar/data/109087...,0001564590-19-032476.txt,Health Care,Health Care,0001564590-19-032476.txt : 20190816 0001564590...,2019-08-16 16:30:14,[]
8,8,8,A,1090872.0,https://www.sec.gov/Archives/edgar/data/109087...,0001090872-19-000013.txt,Health Care,Health Care,0001090872-19-000013.txt : 20190814 0001090872...,2019-08-14 16:07:17,"['Item 2.02', 'Item 2.02', 'Item 9.01']"
9,9,9,A,1090872.0,https://www.sec.gov/Archives/edgar/data/109087...,0001564590-19-030780.txt,Health Care,Health Care,0001564590-19-030780.txt : 20190808 0001564590...,2019-08-08 16:11:00,[]


In [122]:
#This lets us know the minimum date we need to pull stock prices for
min(master['release_date'])

'2013-02-14 16:00:41'

In [123]:
#This alerts us to errors
max(master['release_date'])

'Passed'

In [124]:
# Only 51 rows had 'passed' as the release date, will remove.
print(len(master[master.release_date == 'Passed']))

51


In [125]:
#Drop rows with passed, or a date in the future set to alert there was a problem
master = master[master.release_date != 'Passed']
master = master[master.release_date != '2050-05-01 10:00:00']

In [126]:
master.head()

,Unnamed: 0,Unnamed: 0.1,ticker,cik,txt_link,doc_name,GICS Sector,GICS Sub Industry,text,release_date,items
0,0,0,A,1090872.0,https://www.sec.gov/Archives/edgar/data/109087...,0001090872-20-000002.txt,Health Care,Health Care,0001090872-20-000002.txt : 20200218 0001090872...,2020-02-18 16:09:58,"['Item 2.02', 'Item 2.02', 'Item 9.01']"
1,1,1,A,1090872.0,https://www.sec.gov/Archives/edgar/data/109087...,0001564590-20-001714.txt,Health Care,Health Care,0001564590-20-001714.txt : 20200123 0001564590...,2020-01-23 08:15:23,[]
2,2,2,A,1090872.0,https://www.sec.gov/Archives/edgar/data/109087...,0001090872-19-000020.txt,Health Care,Health Care,0001090872-19-000020.txt : 20191125 0001090872...,2019-11-25 16:07:24,"['Item 2.02', 'Item 2.02', 'Item 9.01']"
3,3,3,A,1090872.0,https://www.sec.gov/Archives/edgar/data/109087...,0001564590-19-037386.txt,Health Care,Health Care,0001564590-19-037386.txt : 20191022 0001564590...,2019-10-22 16:05:46,[]
4,4,4,A,1090872.0,https://www.sec.gov/Archives/edgar/data/109087...,0001564590-19-034970.txt,Health Care,Health Care,0001564590-19-034970.txt : 20190918 0001564590...,2019-09-18 16:30:38,[]


### Load Financial Data

In [127]:
#Please see the other notebook for the code used to download these stock prices.
#These were downloaded using a webscraper, and then combined into one file.  
#The movement for each day is the difference between the adjusted close price for the previous trading day
#I used excel to create the additional features for these, the sheet with the formulas will be in the github.
stock_prices = pd.read_csv("F:\Files from Linux\Stock movements\stock historical data.csv")

In [128]:
#Make sure date is convereted to pandas date for the stock prices
stock_prices['Date']  = pd.to_datetime(stock_prices['Date'])

In [129]:
#pull just the date out of the release_date convert it pandas datetime
master['release_date_day'] = master['release_date'].str[:10]
master['release_date']  = pd.to_datetime(master['release_date'])
#Create a new variable consisting of just the day, we will use this to merge the dfs later
master['release_date_day'] = pd.to_datetime(master['release_date_day'])

In [130]:
#identify if the release was done afterhours (4pm)
#Done by pulling the hour out of the time stamp
master['afterhours'] = np.where(master['release_date'].dt.hour>15, 1, 0)

In [131]:
#if it was released afterhours, incremet the release date day by 1
#if the adjusted day is a sunday, increment by 1, if a saturday increment it by 2
#this is because there is no trading on weekends
master['adj_day'] = np.where(master['afterhours']==1, master['release_date_day'] + datetime.timedelta(days=1) ,master['release_date_day'])
master['adj_day'] = np.where(master['adj_day'].dt.dayofweek == 6 , master['adj_day'] + datetime.timedelta(days=1) ,master['adj_day'])
master['adj_day'] = np.where(master['adj_day'].dt.dayofweek == 5 , master['adj_day'] + datetime.timedelta(days=2) ,master['adj_day'])

In [132]:
#max day of the week is a friday as intended
print(min(master['adj_day'].dt.dayofweek))
print(max(master['adj_day'].dt.dayofweek))

0
4


In [133]:
#Here we show the change in the S&p 500 that day, the change in the adjusted last closing price
#The change in the stock v the change in the S&P 
#and the classification (Positive 1% or more, Negative-1% or less, Neutral -.999% to .999%)
stock_prices.head()

,Date,High,Low,Open,Close,Volume,Adj Close,Name,ChangevLastadjustedclose,S&Pmovement,ChangeVS&P500,Classification
0,2012-02-29,7.62,7.27,7.31,7.41,10447500,6.985714,AAL,0.000000,0.00000,0.000000,Neutral
1,2012-03-01,7.60,7.28,7.45,7.41,7718600,6.985714,AAL,0.000000,0.00000,0.000000,Neutral
2,2012-03-02,7.83,7.39,7.45,7.63,7752700,7.193117,AAL,0.029690,0.00000,0.029690,Positive
3,2012-03-05,7.39,6.96,7.39,6.99,15562100,6.589763,AAL,-0.083879,0.00000,-0.083879,Negative
4,2012-03-06,7.25,6.85,6.92,6.88,7993300,6.486061,AAL,-0.015737,0.01537,-0.031107,Negative


In [134]:
#Join table based on ticker and date
master = pd.merge(left=master, right = stock_prices, how ='left', left_on=['ticker','adj_day'], right_on=['Name','Date'])

In [288]:
#this was start of code to identify if the adjusted date ended up being on a holiday.  
#I decided not to implement it based on the minimum number possibly affected (around 400)
#master['adj_day'] = np.where(len(master['ChangeVS&P500']) == 0 , master['adj_day'] + datetime.timedelta(days=1) ,master['adj_day'])

In [135]:
#for some reason its throwing an error when i try to use the word classification
#renaming it to target
master['target'] = master.iloc[:,-1]

In [136]:
#Create DF of rows missing values to assist in troubleshooting
missing = master[master.isnull().any(axis=1)]

In [138]:
#470 are missing out of about 20,000, acceptable
#likely due to holidays
print(len(missing))

470


In [137]:
#Initial stock prices pulled were missing some, this was used to identify them
#Some are still missing but in acceptable amounts (about 400 out of 20,000)
missing['ticker'].value_counts()

BF.B     40
BRK.B    40
ABMD     40
FOX      32
FOXA     31
NLOK     21
STE       7
C         4
PYPL      4
PLD       3
RHI       3
EXPD      3
MS        3
MET       3
VMC       3
LW        3
TWTR      3
ATVI      3
BLK       3
HUM       3
ARNC      3
AZO       3
CBOE      3
EXR       2
CHTR      2
DOW       2
TAP       2
INFO      2
URI       2
EVRG      2
         ..
HBAN      1
CSCO      1
MU        1
HSIC      1
GWW       1
FFIV      1
SJM       1
EIX       1
STT       1
TIF       1
MGM       1
FDX       1
FLS       1
CHD       1
OMC       1
FAST      1
CNC       1
IVZ       1
VNO       1
CLX       1
CTVA      1
XEL       1
NKE       1
AVY       1
D         1
DGX       1
IQV       1
SBUX      1
MCK       1
LYB       1
Name: ticker, Length: 192, dtype: int64

In [139]:
print(len(master))

19921


In [140]:
#Drop NA values
#Drop Index columns that creeped in
#Drop low, open, close,name, and low to lower size of output file
master = master[master['target'].notna()]
master.drop(master.iloc[:, 0:2], inplace = True, axis = 1) 
master.drop(['Open','Close','Name','High',"Low"], inplace = True, axis = 1) 

In [149]:
master.head()

,ticker,cik,txt_link,doc_name,GICS Sector,GICS Sub Industry,text,release_date,items,release_date_day,afterhours,adj_day,Date,Volume,Adj Close,ChangevLastadjustedclose,S&Pmovement,ChangeVS&P500,Classification,target
0,A,1090872.0,https://www.sec.gov/Archives/edgar/data/109087...,0001090872-20-000002.txt,Health Care,Health Care,0001090872-20-000002.txt : 20200218 0001090872...,2020-02-18 16:09:58,"['Item 2.02', 'Item 2.02', 'Item 9.01']",2020-02-18,1,2020-02-19,2020-02-19,4741300.0,85.349998,0.006605,-0.004706,0.011310,Positive,Positive
1,A,1090872.0,https://www.sec.gov/Archives/edgar/data/109087...,0001564590-20-001714.txt,Health Care,Health Care,0001564590-20-001714.txt : 20200123 0001564590...,2020-01-23 08:15:23,[],2020-01-23,0,2020-01-23,2020-01-23,1228300.0,89.070000,-0.011761,-0.001141,-0.010620,Negative,Negative
2,A,1090872.0,https://www.sec.gov/Archives/edgar/data/109087...,0001090872-19-000020.txt,Health Care,Health Care,0001090872-19-000020.txt : 20191125 0001090872...,2019-11-25 16:07:24,"['Item 2.02', 'Item 2.02', 'Item 9.01']",2019-11-25,1,2019-11-26,2019-11-26,5329900.0,80.779411,0.008597,-0.002196,0.010793,Positive,Positive
3,A,1090872.0,https://www.sec.gov/Archives/edgar/data/109087...,0001564590-19-037386.txt,Health Care,Health Care,0001564590-19-037386.txt : 20191022 0001564590...,2019-10-22 16:05:46,[],2019-10-22,1,2019-10-23,2019-10-23,1778300.0,74.821999,0.003883,-0.002847,0.006730,Neutral,Neutral
4,A,1090872.0,https://www.sec.gov/Archives/edgar/data/109087...,0001564590-19-034970.txt,Health Care,Health Care,0001564590-19-034970.txt : 20190918 0001564590...,2019-09-18 16:30:38,[],2019-09-18,1,2019-09-19,2019-09-19,1743700.0,77.738228,0.006446,-0.000020,0.006466,Neutral,Neutral


In [150]:
#Nas dropped
print(len(master))

19451


In [154]:
#Save file to be used in next notebook
master.to_csv("F:\Files from Linux\Raw Texts.textcorpus.csv",header=True)